In [ ]:
!pip install matplotlib
!pip install scikit-learn
!pip install tensorflow
!pip install keras==2.12.0

In [ ]:
import requests
import time
import sys
import json
import pickle
import numpy as np
import pandas as pd
from google.colab import userdata
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Testing on Raw Data

In [ ]:
with open("/content/drive/MyDrive/ML/dataset.pkl", 'rb') as file:
  data = pickle.load(file)

In [ ]:
X_blue = []
X_red = []
y = []
team_indicator_blue = []
team_indicator_red = []

In [ ]:
for i in range(10001):
    blue_side_features = []
    red_side_features = []
    blue_side_indicator = [0] * 5  # 0 for blue side
    red_side_indicator = [1] * 5   # 1 for red side

    matchData = data[i]
    for i, participant in enumerate(matchData):
        # Extract features
        #print(participant)
        champion_id = participant["championId"]

        # Decide which team (blue or red) the participant belongs to
        if i < 5:
            blue_side_features.append(champion_id)
        else:
            red_side_features.append(champion_id)

    y.append(1 if participant["win"] and i >= 5 else 0)
    X_blue.append(blue_side_features)
    X_red.append(red_side_features)
    team_indicator_blue.append(blue_side_indicator)
    team_indicator_red.append(red_side_indicator)

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Convert lists to NumPy arrays
X_blue = np.array(X_blue)
X_red = np.array(X_red)
y = np.array(y)

# Create a OneHotEncoder
encoder = OneHotEncoder(categories='auto', sparse=False)

# Fit and transform champion IDs for both blue and red teams
X_blue_encoded = encoder.fit_transform(X_blue)
X_red_encoded = encoder.fit_transform(X_red)

# Concatenate blue and red features along the last axis
X_encoded = np.concatenate((X_blue_encoded, X_red_encoded), axis=-1)

# Concatenate team indicators
team_indicator_blue = np.array(team_indicator_blue)
team_indicator_red = np.array(team_indicator_red)
team_indicators = np.concatenate((team_indicator_blue, team_indicator_red), axis=-1)

# Combine features and team indicators
X_combined = np.concatenate((X_encoded, team_indicators), axis=-1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=21)

# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(200, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(60, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")

For raw data only, the highest accuracy we were able to get was 54%

### Testing on Engineered Features

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ML/data.csv")
df

In [ ]:
from os import X_OK
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = df.drop("teamWin", axis=1)

y = df["teamWin"]

In [ ]:
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

def create_model(hidden_units=(16,), activation='relu', optimizer='adam', dropout_rate=0.0, weight_decay=0.0):
    model = Sequential()
    for units in hidden_units:
        model.add(Dense(units=units, activation=activation, kernel_regularizer='l2'))
        model.add(Dropout(dropout_rate))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create a KerasClassifier for use with scikit-learn GridSearchCV
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

# Define hyperparameters and their possible values
param_grid = {
    'hidden_units': [[50], [20, 10, 6], [200, 100, 60], [64, 32, 16, 8]],
    'activation': ['relu', 'sigmoid'],
    'optimizer': ['adam', 'rmsprop'],
    'dropout_rate': [0.0, 0.2, 0.4],
    'weight_decay': [0.0, 1e-5, 1e-4]
}

# Use GridSearchCV to perform the search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train_scaled, y_train)

# Print the best parameters and corresponding accuracy
print("Best Parameters: ", grid_result.best_params_)
print("Best Accuracy: {:.2f}%".format(grid_result.best_score_ * 100))

# Evaluate the model on the test set with the best parameters
best_model = grid_result.best_estimator_
y_pred = best_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy with Best Parameters: {:.2f}%".format(accuracy * 100))


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
param_dist = {
    'n_estimators': [50, 100, 200, 300, 500],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy'],
    'min_weight_fraction_leaf': [0.0, 0.1, 0.2],
    'max_leaf_nodes': [None, 10, 20, 30],
}

# Create a Random Forest classifier
rf_model = RandomForestClassifier()

# Perform randomized search
random_search = RandomizedSearchCV(
    rf_model, param_distributions=param_dist, scoring='accuracy', n_iter=100, cv=3, verbose=1, random_state=21
)
random_search.fit(X_train, y_train)

# Summarize results
print(f"Best accuracy: {random_search.best_score_} using {random_search.best_params_}")

# Evaluate the best model on the test set
best_rf_model = random_search.best_estimator_
y_pred = best_rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test set accuracy: {accuracy}")

In [ ]:
import matplotlib.pyplot as plt

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

# Create and train a Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=21)
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the model accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 68.10%


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Standardize the input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the number of folds for cross-validation
n_folds = 10  # You can adjust this based on your preference

# Initialize StratifiedKFold
stratified_kfold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# Create a neural network model
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(50, activation='relu', input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize an empty list to store accuracy scores
accuracy_scores = []

# Perform cross-validation
for train_index, test_index in stratified_kfold.split(X_scaled, y):
    X_train, X_test = X_scaled[train_index], X_scaled[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Create and train the model
    model = create_model()
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Evaluate the model on the test set
    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype(int)

    # Calculate and store the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

# Print the accuracy scores for each fold
for i, accuracy in enumerate(accuracy_scores, start=1):
    print(f"Fold {i} accuracy: {accuracy}")

# Print the mean and standard deviation of the accuracy scores
print(f"Mean accuracy: {np.mean(accuracy_scores)}")
print(f"Standard deviation: {np.std(accuracy_scores)}")


### Saving the best model to drive

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from google.colab import drive

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

# Standardize the input features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Best parameters from your grid search
best_params = {'activation': 'relu', 'dropout_rate': 0.4, 'hidden_units': [50], 'optimizer': 'rmsprop', 'weight_decay': 0.0001}

# Create the model with the best parameters
best_model = Sequential()
for units in best_params['hidden_units']:
    best_model.add(Dense(units=units, activation=best_params['activation'], kernel_regularizer='l2'))
    best_model.add(Dropout(best_params['dropout_rate']))
best_model.add(Dense(units=1, activation='sigmoid'))
best_model.compile(loss='binary_crossentropy', optimizer=best_params['optimizer'], metrics=['accuracy'])

# Train the model with the entire training set
best_model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=0)

# Save the best model to Google Drive
best_model.save("/content/drive/My Drive/ML/model.h5")

# Evaluate the model on the test set
y_pred = best_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred.round())
print("Test Accuracy with Best Parameters: {:.2f}%".format(accuracy * 100))
